In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
df = pd.read_csv("tcc_ceds_music.csv")
df = df.drop('Unnamed: 0',axis=1)

In [3]:
df

,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.000000
1,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,0.001284,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.000000
2,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.000000
3,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.000000
4,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,0.001350,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,78,0.001350,0.001350,0.001350,0.001350,...,0.065664,0.001350,0.889527,0.759711,0.062549,0.000000,0.751649,0.695686,obscene,0.014286
28368,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,67,0.001284,0.001284,0.035338,0.001284,...,0.001284,0.001284,0.662082,0.789580,0.004607,0.000002,0.922712,0.797791,obscene,0.014286
28369,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,77,0.001504,0.154302,0.168988,0.001504,...,0.001504,0.001504,0.663165,0.726970,0.104417,0.000001,0.838211,0.767761,obscene,0.014286
28370,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,67,0.001196,0.001196,0.001196,0.001196,...,0.001196,0.001196,0.883028,0.786888,0.007027,0.000503,0.508450,0.885882,obscene,0.014286


In [4]:
features = ['len', 'dating', 'violence', 'world/life', 'night/time',
       'shake the audience', 'family/gospel', 'romantic', 'communication',
       'obscene', 'music', 'movement/places', 'light/visual perceptions',
       'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability',
       'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy', 'age']

In [5]:
for feature in features:
    df[feature] = pd.qcut(df[feature], q=3, labels=["low","medium","high"])

In [6]:
df

,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,high,low,medium,low,low,...,high,high,low,low,high,high,low,low,sadness,high
1,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,medium,high,medium,high,medium,...,low,medium,low,medium,high,low,low,low,world/life,high
2,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,low,high,medium,medium,medium,...,medium,high,low,low,high,low,low,low,music,high
3,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,medium,high,medium,medium,medium,...,high,medium,high,high,medium,high,high,high,romantic,high
4,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,low,medium,low,high,medium,...,medium,medium,low,medium,high,medium,medium,low,romantic,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28367,mack 10,10 million ways,2019,hip hop,cause fuck leave scar tick tock clock come kno...,medium,medium,low,low,medium,...,medium,medium,high,high,low,low,high,medium,obscene,low
28368,m.o.p.,ante up (robbin hoodz theory),2019,hip hop,minks things chain ring braclets yap fame come...,medium,medium,low,medium,medium,...,low,medium,high,high,low,low,high,high,obscene,low
28369,nine,whutcha want?,2019,hip hop,get ban get ban stick crack relax plan attack ...,medium,medium,high,high,medium,...,medium,medium,high,medium,medium,low,high,high,obscene,low
28370,will smith,switch,2019,hip hop,check check yeah yeah hear thing call switch g...,medium,medium,low,low,low,...,low,medium,high,high,low,medium,medium,high,obscene,low


In [7]:
#list(set(df["artist_name"].unique()))

In [18]:
def construct_graph(df_, features_):
    features = features_.copy()
    df = df_.copy()
    
    graph = nx.MultiGraph()
    
    for genre in list(set(df["genre"].unique())):
        graph.add_node(f"g:{genre}", ntype='category')
    
    for artist in list(set(df["artist_name"].unique())):
        graph.add_node(f"a:{artist}", ntype='category')
    
    for feature in features:
        graph.add_node(f"{feature}-low", ntype='category')
        graph.add_node(f"{feature}-medium", ntype='category')
        graph.add_node(f"{feature}-high", ntype='category')
        
    for idx, row in df.iterrows():
        graph.add_node(f"{idx}-song", ntype='song', name=row['track_name'], artist=row['artist_name'], genre=row["genre"])
        graph.add_edge(f"{idx}-song", f"a:{row['artist_name']}",weight = 0.001)
        graph.add_edge(f"{idx}-song", f"g:{row['genre']}", weight = 0.01)
        
        for feature in features:
            if (feature == 'danceability'):
                graph.add_edge(f"{feature}-{row[feature]}",f"{idx}-song",weight = 2)
            else:
                graph.add_edge(f"{feature}-{row[feature]}",f"{idx}-song",weight = 0.9)

        
    return graph

In [19]:
G = construct_graph(df, features)

In [20]:
len(G.nodes())

33877

In [21]:
lccs = list(nx.connected_components(G))
len(lccs)

1

In [22]:
def continue_playlist(personalization_dict, plength=30):
    
    rankings = nx.pagerank(G, personalization=personalization_dict, weight='weight')

    rankings = [t for t in sorted(rankings.items(), key=lambda x: x[1], reverse=True)]
    
    playlist = []
    i = 0
    for t in rankings:
        if nx.get_node_attributes(G,'ntype')[t[0]] == 'song':
            playlist.append(f"{nx.get_node_attributes(G,'name')[t[0]]} - {nx.get_node_attributes(G,'artist')[t[0]]} - {nx.get_node_attributes(G,'genre')[t[0]]} --- {t[0]}")
            i+=1
            if i >=plength:
                break
    
    return playlist

In [23]:
personalization_dict = {
    '27467-song': 1, #Imagine dragons - birds
    '23520-song': 1, #The kingston trio - the long black rifle
    '24077-song': 1, #Eagles - I wish you peace
}

continue_playlist(personalization_dict)

['i wish you peace - eagles - rock --- 24077-song',
 'the long black rifle - the kingston trio - rock --- 23520-song',
 'birds (feat. elisa) - imagine dragons - rock --- 27467-song',
 'i wish it had been a dream - don gibson - country --- 7241-song',
 'a wonderful yesterday - willie nelson - country --- 8203-song',
 'sleep song - graham nash - country --- 8433-song',
 "until it's time for you to go - willie nelson - country --- 9572-song",
 'so long - ruth brown - blues --- 12657-song',
 "until it's time for you to go - neil diamond - pop --- 1774-song",
 'night and day - etta james - blues --- 12879-song',
 'and further on - jethro tull - blues --- 14103-song',
 'wonderful future - willie nelson - country --- 8459-song',
 'forgive me - pat boone - rock --- 23439-song',
 'i walk alone - marty robbins - country --- 7078-song',
 "for heaven's sake - billie holiday - blues --- 12676-song",
 'cry and you cry alone - lesley gore - pop --- 644-song',
 'that old feeling - the platters - pop -

In [24]:
personalization_dict = {
    '5082-song': 1, #britney spears - breathe on me
    '4598-song': 1, #madonna - nothing really matters
    '1842-song': 1, #abba - waterloo
}

continue_playlist(personalization_dict)

['breathe on me - britney spears - pop --- 5082-song',
 'nothing really matters - madonna - pop --- 4598-song',
 'waterloo - abba - pop --- 1842-song',
 'strangered in the night - tom petty and the heartbreakers - pop --- 2134-song',
 'frisk me down - katchafire - reggae --- 23122-song',
 'through the never - metallica - rock --- 25356-song',
 'but i feel good - groove armada - jazz --- 19724-song',
 "(i'm a) road runner - jr. walker & the all stars - pop --- 948-song",
 'mouth of the river - imagine dragons - rock --- 27277-song',
 'daytime drama - billy idol - pop --- 2975-song',
 'guns out - iration - reggae --- 23156-song',
 "can't take that away - peter frampton - country --- 10618-song",
 'the three bells (aka jimmy brown) - ken parker - reggae --- 23015-song',
 'tomorrow comes today - gorillaz - pop --- 4910-song',
 "let's kiss & say goodbye - john holt - reggae --- 21178-song",
 'currency - the black angels - blues --- 16847-song',
 'who needs sleep? - barenaked ladies - pop --

In [25]:
personalization_dict = {
    '6037-song': 1, #avicii - levels,
    '6058-song': 1, #david guetta - titanium (feat. sia)
}

continue_playlist(personalization_dict)

['levels - avicii - pop --- 6037-song',
 'titanium (feat. sia) - david guetta - pop --- 6058-song',
 'alive - korn - rock --- 26219-song',
 'i want it all - arctic monkeys - rock --- 26926-song',
 'third law - leprous - jazz --- 20576-song',
 "you'll never find me - korn - rock --- 27461-song",
 'an autopsy - the faceless - jazz --- 19960-song',
 'imprint of the un-saved - meshuggah - jazz --- 19887-song',
 'happy pills - weathers - rock --- 27176-song',
 'warrant - foster the people - rock --- 26711-song',
 'spectrum - zedd - pop --- 6175-song',
 'shoot the runner - kasabian - rock --- 26402-song',
 'dirthouse - static-x - rock --- 26339-song',
 'martyrs - soen - jazz --- 20886-song',
 'if all goes well - the dear hunter - jazz --- 20604-song',
 'voodoo mon amour - diablo swing orchestra - jazz --- 20415-song',
 'new theory - washed out - rock --- 26608-song',
 'breakbeat - slightly stoopid - reggae --- 22620-song',
 'give it all - rise against - pop --- 5299-song',
 'bury me with my 

In [17]:
df[df["artist_name"] == "snarky puppy"]

,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
20435,snarky puppy,gone under,2013,jazz,heart bleed break thing hold alltime sleep eye...,high,low,medium,low,low,...,high,high,medium,medium,high,low,medium,medium,sadness,low
20454,snarky puppy,too hot to last,2013,jazz,friend hard end die begin blame past blame cir...,low,medium,high,medium,medium,...,high,medium,low,low,high,high,low,low,sadness,low
20935,snarky puppy,bardis,2019,jazz,look depth perception cause hurt deeper think ...,low,medium,high,medium,high,...,high,medium,medium,medium,low,high,low,medium,sadness,low


In [ ]:
personalization_dict = {
    '20435-song': 1, #snarky puppy - gone under
    '6058-song': 1, #david guetta - titanium (feat. sia)
}

continue_playlist(personalization_dict)